# 10+ Minutes to Dask

<a href="https://colab.research.google.com/github/shauryashaurya/learn-data-munging/blob/main/04-Dask/01.01-10%2B-minutes-to-dask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Intro

Dask builds upon it's own data-objects: DataFrames, Arrays and Dask Bag.  
We'll tackle each in turn.

## Setup

In [1]:
import dask
import dask.array as da
import dask.bag as db
import dask.dataframe as dd
import numpy as np
import pandas as pd

In [ ]:
from dask.distributed import Client, LocalCluster
client = Client()

client

# Dask Objects

## Dask DataFrames

Dask Dataframes coordinate many Pandas dataframes, partitioned along an index.  
Support a subset of the Pandas API.  


In [11]:
# dask dataframe
# from pandas
idx = pd.date_range("2023-05-06", periods=1000, freq="1H")

In [12]:
idx

DatetimeIndex(['2023-05-06 00:00:00', '2023-05-06 01:00:00',
               '2023-05-06 02:00:00', '2023-05-06 03:00:00',
               '2023-05-06 04:00:00', '2023-05-06 05:00:00',
               '2023-05-06 06:00:00', '2023-05-06 07:00:00',
               '2023-05-06 08:00:00', '2023-05-06 09:00:00',
               ...
               '2023-06-16 06:00:00', '2023-06-16 07:00:00',
               '2023-06-16 08:00:00', '2023-06-16 09:00:00',
               '2023-06-16 10:00:00', '2023-06-16 11:00:00',
               '2023-06-16 12:00:00', '2023-06-16 13:00:00',
               '2023-06-16 14:00:00', '2023-06-16 15:00:00'],
              dtype='datetime64[ns]', length=1000, freq='H')

In [13]:
pd_df = pd.DataFrame({"a": np.arange(1000), "b": list("abcd" * 250)}, index=idx)

In [14]:
pd_df

,a,b
2023-05-06 00:00:00,0,a
2023-05-06 01:00:00,1,b
2023-05-06 02:00:00,2,c
2023-05-06 03:00:00,3,d
2023-05-06 04:00:00,4,a
...,...,...
2023-06-16 11:00:00,995,d
2023-06-16 12:00:00,996,a
2023-06-16 13:00:00,997,b
2023-06-16 14:00:00,998,c


In [15]:
dask_df = dd.from_pandas(pd_df, npartitions=10)

In [16]:
dask_df

,a,b
npartitions=10,,
2023-05-06 00:00:00,int32,string
2023-05-10 04:00:00,...,...
...,...,...
2023-06-12 12:00:00,...,...
2023-06-16 15:00:00,...,...


In [17]:
dask_df.divisions

(Timestamp('2023-05-06 00:00:00'),
 Timestamp('2023-05-10 04:00:00'),
 Timestamp('2023-05-14 08:00:00'),
 Timestamp('2023-05-18 12:00:00'),
 Timestamp('2023-05-22 16:00:00'),
 Timestamp('2023-05-26 20:00:00'),
 Timestamp('2023-05-31 00:00:00'),
 Timestamp('2023-06-04 04:00:00'),
 Timestamp('2023-06-08 08:00:00'),
 Timestamp('2023-06-12 12:00:00'),
 Timestamp('2023-06-16 15:00:00'))

In [18]:
dask_df.partitions[1]

,a,b
npartitions=1,,
2023-05-10 04:00:00,int32,string
2023-05-14 08:00:00,...,...


In [19]:
# data types of each of the columns
dask_df.dtypes

a              int32
b    string[pyarrow]
dtype: object

We can do regular Pandas stuff with Dask Dataframes now...

In [20]:
# get a subset based on index (date-time)
dask_df2 = dask_df.loc[idx[0:100]]

In [21]:
dask_df2

,a,b
npartitions=1,,
2023-05-06 00:00:00,int32,string
2023-05-10 03:00:00,...,...


In [22]:
# perform analysis on the subset
dask_df2_grpby_count = dask_df2.groupby("b").count()

In [23]:
# Dask evaluates lazy
# nothing happens untill we call .compute()
dask_df2_grpby_count.compute()

,a
b,
a,25
b,25
c,25
d,25


## Dask Arrays

Dask arrays coordinate many Numpy arrays, arranged into chunks within a grid.  
Dask arrays support a subset of Numpy API.

In [24]:
np_array = np.arange(100000).reshape(200, 500)

In [25]:
dask_array = da.from_array(np_array, chunks=(100, 100))

In [26]:
dask_array

dask.array<array, shape=(200, 500), dtype=int32, chunksize=(100, 100), chunktype=numpy.ndarray>

In [27]:
dask_array.chunks

((100, 100), (100, 100, 100, 100, 100))

In [28]:
dask_array.blocks[1, 3]

dask.array<blocks, shape=(100, 100), dtype=int32, chunksize=(100, 100), chunktype=numpy.ndarray>

In [29]:
# let's play with a slightly more interesting example
# x is a matrix of random numbers
x = da.random.random((100, 100), chunks=(10, 10))

In [30]:
x

dask.array<random_sample, shape=(100, 100), dtype=float64, chunksize=(10, 10), chunktype=numpy.ndarray>

In [31]:
# operations just like Numpy
y = x + x.T
y

dask.array<add, shape=(100, 100), dtype=float64, chunksize=(10, 10), chunktype=numpy.ndarray>

In [32]:
z1 = y[::2, 50:].mean(axis=0)
z2 = y[::2, 50:].mean(axis=1)

In [33]:
z1

dask.array<mean_agg-aggregate, shape=(50,), dtype=float64, chunksize=(10,), chunktype=numpy.ndarray>

In [34]:
# to actually compute z1, let's use .compute()
z1.compute()

array([1.11492005, 1.05305538, 1.03071252, 1.15530144, 1.04823263,
       1.06198146, 1.06484971, 1.00576768, 1.07294368, 1.02853164,
       0.95526389, 1.05540597, 1.09882387, 1.10431726, 1.10649438,
       1.06950176, 1.09897933, 1.03173242, 0.95644992, 0.96571605,
       0.97805217, 1.11324138, 0.98590791, 0.99101214, 0.99619895,
       0.92278542, 1.05755604, 1.00297909, 1.01589087, 0.93217715,
       1.07566113, 1.04352775, 1.00566876, 1.03585958, 0.96835403,
       0.99550083, 1.02056448, 1.01689741, 1.01026697, 0.90892673,
       1.0199655 , 0.96206546, 1.01098322, 1.07061565, 1.07299468,
       1.03107508, 1.06199941, 0.99552828, 0.9754062 , 0.95361026])

In [26]:
z2

dask.array<mean_agg-aggregate, shape=(50,), dtype=float64, chunksize=(5,), chunktype=numpy.ndarray>

In [27]:
z2.compute()

array([1.06705481, 0.98787893, 1.10951139, 1.02256976, 0.96535061,
       1.02407932, 1.01761317, 0.96132722, 1.02600042, 1.17221817,
       0.97977577, 0.89607627, 1.05186492, 0.91498646, 1.00499123,
       0.94766529, 1.08152637, 1.02423327, 0.9202498 , 1.01481534,
       1.07190135, 0.95071588, 1.09227694, 0.99321922, 0.9766521 ,
       0.99253299, 0.94190823, 1.08216132, 0.89454637, 0.90053082,
       0.9974013 , 1.02626321, 0.96902153, 0.95935884, 0.9203113 ,
       0.92836323, 0.99664067, 1.00653366, 0.89564659, 1.03732168,
       1.07595559, 0.96889028, 1.03185385, 1.0109487 , 1.00655903,
       0.97521872, 0.92876645, 0.9879    , 0.99389869, 1.07673466])

## Dask Bag

Bag is unordered collection of objects allowing repeats. Use these for semi/un-structured data.  
It's fun but slower than dataframes and arrays.  
The [examples](https://examples.dask.org/bag.html) page is really interesting.

In [28]:
dask_bag = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 0], npartitions=2)

In [29]:
dask_bag

dask.bag<from_sequence, npartitions=2>

In [30]:
dask_bag.take(2)

(1, 2)

In [31]:
# dask is lazy - this one grabs values from one partition
dask_bag.filter(lambda x: x > 3).take(2)

(4, 5)

In [32]:
# Here's how we take ALL across all partitions
dask_bag.filter(lambda x: x > 3).compute()

[4, 5, 6, 7, 8, 9]

In [33]:
dask_bag.map(lambda x: x * x).take(5)

(1, 4, 9, 16, 25)

In [34]:
dask_bag.count().compute()

10

In [35]:
# convert to a dask dataframe
# this is a trivial example
dask_df_from_bag = dask_bag.to_dataframe()

In [36]:
dask_df_from_bag

,0
npartitions=2,
,int64
,...
,...


### Build bag with complex json and convert to dataframe
* Step 1: define a 'flatten' function
* Step 2: map 'flatten' to the bag
* Step 3: convert the flattened bag to dataframe using bag_instance.to_dataframe()

Using example from https://examples.dask.org/bag.html

#### Create Random Data

In [37]:
import json
import os

In [38]:
os.makedirs("./data/dask-bag-example-01", exist_ok=True)

In [39]:
b = dask.datasets.make_people()

In [ ]:
b.map(json.dumps).to_textfiles("./data/dask-bag-example-01/*.json")

#### Read JSON Data

In [41]:
# for windows
# !more .\data\dask-bag-example-01\0.json
# for linux
# !head -n 2 ./data/dask-bag-example-01/0.json

In [42]:
b = db.read_text("./data/dask-bag-example-01/*.json").map(json.loads)
b

dask.bag<loads, npartitions=10>

In [43]:
b.take(2)

({'age': 43,
  'name': ['Buck', 'Graham'],
  'occupation': 'Interior Decorator',
  'telephone': '+14425239000',
  'address': {'address': '879 Bellevue Canyon', 'city': 'Independence'},
  'credit-card': {'number': '5136 3169 4754 0304',
   'expiration-date': '07/16'}},
 {'age': 27,
  'name': ['Zonia', 'Montgomery'],
  'occupation': 'Stage Director',
  'telephone': '+1-956-696-7586',
  'address': {'address': '406 Converse Rapids', 'city': 'Pearland'},
  'credit-card': {'number': '4178 5897 3296 0786',
   'expiration-date': '08/22'}})

In [ ]:
client.retire_workers()
client.shutdown()